# Set up environment

In [ ]:
# Install Foundry
!pip install foundry-ml pandas==1.3.5

In [ ]:
# Import some generally useful modules
from dlhub_sdk.client import DLHubClient
from foundry import Foundry
import pandas as pd

# Data+Model Template: Diffusion data and model

## Get and show diffusion data



* Dataset 19
* Relevant reference: H. J. Lu, N. Zou, R. Jacobs, B. Afflerbach, X. G. Lu, and D. Morgan, Error assessment and optimal cross-validation approaches in machine learning applied to impurity diffusion, Computational Materials Science 169 (July) (2019). https://doi.org/10.1016/j.commatsci.2019.06.010
* Data DOI: 10.18126/uppe-p8p1 


In [ ]:
# Load and print out data
f = Foundry(index="mdf")
data = f.load('diffusion_v1.4', globus=False).load_data()
print(data)
X = data['train'][0]  # Features pre-generation
y = data['train'][1]  # Target variable
print(X)
print(y)

## Get and show diffusion model
* Name: Ryan Jacobs
* Email: rjacobs3@wisc.edu
* Relevant reference: H. J. Lu, N. Zou, R. Jacobs, B. Afflerbach, X. G. Lu, and D. Morgan, Error assessment and optimal cross-validation approaches in machine learning applied to impurity diffusion, Computational Materials Science 169 (July) (2019). https://doi.org/10.1016/j.commatsci.2019.06.010
* Date added/last modified in this notebook: 10/18/22
* Foundry hosted data link: https://foundry-ml.org/#/datasets/10.18126%2Fuppe-p8p1
* Foundry hosted data DOI: 10.18126/uppe-p8p1 

In [ ]:
# Set servable to work with
# Note that this first line should generate a link and the link "Please Paste your Auth Code Below:" with an box to enter your code from the link.
dlhub_servable = 'rjacobs3_wisc/MASTML_Tutorial_Model_DLHub_v100_2022-10-13'
# Pring out servable info
desc=DLHubClient().describe_servable(dlhub_servable)
#desc["servable"].keys()
print('Servible Name (if availble): ', desc['dlhub']['shorthand_name'])
print('Servible Datacite (if availble): ', desc['datacite']['identifier']['identifier'])
print('Servible DOI (if availble): ', desc['datacite']['identifier']['identifierType'])

In [ ]:
# This model takes as input 2 elements, the first is the host and the second is an impurity, and it returns the activation energy for impurity diffusion.
# For example, the input CuAg gives the activation energy of dilute Ag in a Cu host.
X_test = pd.DataFrame({'Material compositions joined': ['CuAl', 'NiAl', 'HeH', 'NiSi', 'PtPd']})
# Info on input format
# DLHubClient().describe_servable(dlhub_servable)
input_dict = {'X_test': X_test, #X_test_Ag_nofeatures
              'X_test_extra': X_test, #X_extra_Ag
             'featurize': True,
             'featurizer': 'ElementalFeatureGenerator',
             'featurize_on': 'Material compositions joined',
              'composition_df': X_test['Material compositions joined'] #X_test_Ag_nofeatures['Material compositions joined']
             }
#dlhub_servable = MY_USERNAME+'/'+model_name
pred_df = DLHubClient().run(name=dlhub_servable, inputs=input_dict, debug=False, validate_input=False)
pred_df